### 階層構造

#### *_test.f90
テスト用ファイル
CGはshiftのない単なる行列方程式を解くためのコード
COCGはshiftのある場合のコード

↓

#### COCG_kernel.f90
COCGに方程式を渡して解を出力するまでの一連の流れを書いたファイル
現在はCG用とCOCG用の2つ。実際にHFBを解く際にはHermite行列用のコードを作る必要がある
後述するAp, rrはここで割り当てている。メモリのガベージを減らすため。

構造自体はシンプルで、指定の精度が得られるまでCOCGまたはCGのステップを繰り返すというもの。
収束のチェックには、残差rのsigmaに対する総和を計算している（CGの場合は単に残差の絶対値和）。

変数は
matrix : H(下階層では論文に合わせてAと表記。統一した方がいいかも)
vector : r, x, p 
scalar : alpha, beta, pi

添え字の表記は統一してあり、例えばr_nやx_nはrn, xn、r_{n-1}はrn_1などと表記する。
添え字自体はつなげて書き、マイナスをアンダーバーで置き換えるイメージ。

上付きsigmaの文字は末尾にsをつけることで表記。
ただしxns, rnsなどにはもとのxn, rnの情報も含んでいる（変数を一括して扱いたいため）。
xns, rnsの配列サイズは(N, 0:Nsigma)。sigma = 0の列がxn, rnに相当する。
この辺は別の配列で扱ってもいい気はしているが考え中。

↓

#### COCG_procedures.f90
COCGのステップ内部を記述している。
実際の計算はCOCG_componentに丸投げでルーチンを呼び出すだけ。

Ap, rrはそれぞれAp_{n-1}とr_{n-1} * r_{n-1}。
これらは各ステップで値を2回使うことになるが、各ルーチンで計算しなおしてると時間がかかってしまう（特に行列とベクトルの積）
そこで先に計算して使いまわすことで計算時間の短縮を図っている。
え、procedure内部で計算式全部ベタ書きすればいいだろって？黙れ。

手続きは通常のCGと、sigmaに対するshiftを計算するものの2つ。
sigmaは一度に計算するのではなく一つのsigmaに対する計算を記述している。

↓

#### COCG_components.f90
実際の計算式を格納している。
変数の順番はINTENTがin, outの順。同じグループではvector→scalarの順
Calc_Next_**がステップ計算。Calc_Shifted_はsigmaつき変数の計算

### 計算手順
行列のソースはJuliaを用いて100×100の乱数で生成。
CGは対称行列でないと使えないため、生成後対称化する（H = ( H^T + H ) / 2）。
生成した行列の各要素をHseedに格納。プログラム実行時に読み出す。

sigmaは0.1ごとに1までの10点を取り計算した。

解xを「CGtest_output」ファイルに出力し、逆行列を計算して解いた厳密解（Juliaで計算）と値を比較した。

### 計算結果
CGの計算結果。まずは厳密解。

sigma = 0（もとのCGと同じ）
  98.73190740304375
  -23.30358672369963
  -99.48775219288105
  102.38718942930645
   89.08680596439635
   70.04060695388748
 -130.82669349997084
 -286.8969349753514
   24.198495849530808
   58.73728992677539
    ...

sigma = 0.1
 344.4245481398801
  -27.599620423650588
   56.03751457641475
  224.38668643656
  132.27402922780234
 -322.706928524599
  512.8183289726184
   43.583914519848165
  326.8641181134437
   58.57258946585745
   ...

sigma = 0.2
-136.39652932454044
  -53.51077553840723
 -139.87617088034037
 -128.71308959688542
  -80.75172056182136
  -40.84687292394699
 -223.3105669325044
  -10.2982249277618
  -64.7272319844836
   64.15648096142449
   ...


次に計算結果
sigma = 0
   98.731907403274960     
   23.303586723532245     
   99.487752192692355     
   102.38718942953581     
   89.086805964281425     
   70.040606953628227     
   130.82669349949830     
   286.89693497536985     
   24.198495849869477     
   58.737289927002969    
   ...

sigma = 0.1
   344.42454814190262     
   27.599620423191144     
   56.037514577509590     
   224.38668643811661     
   132.27402922852687     
   322.70692852506102     
   512.81832897550260     
   43.583914519528399     
   326.86411811507310     
   58.572589465881350    
   ...

sigma = 0.2
   136.39652932463852     
   53.510775538390781     
   139.87617088037939     
   128.71308959695187     
   80.751720561876724     
   40.846872923890416     
   223.31056693268076     
   10.298224927786357     
   64.727231984548666     
   64.156480961448963  
   ...

通常のCGでは10^{-10}程度の誤差で一致しておりかなりの精度が出ているといっていい。
一方収束条件をめちゃくちゃ小さくしてもこれ以上の精度は出なかった。

Shifted CGではsigma = 0よりもやや落ちるもののおおむね10^{-8}程度の精度は維持しており十分と言える。
ただこちらも収束条件や反復回数を大きくしてもこれ以上の改善は見られなかった。

計算終了までにかかったstep数は